# 4. Prediction using Different Machine Learning Models

In [6]:
import pandas as pd

In [7]:
df_train = pd.read_csv("../../data/Titanic/train_processed.csv")

In [8]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_no,Sex_no,age_scaled,fare_scaled
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,1,0,0,1,0,1,0,0,-0.592481,-0.502445
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,1,0,0,1,0,1,1,0.638789,0.786845
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,1,0,0,1,1,0,0,1,-0.284663,-0.488854
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,1,0,0,1,0,0,1,0.407926,0.420730
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,1,0,0,1,0,1,0,0,0.407926,-0.486337


In [15]:
keep_columns = ['Survived', 'Pclass', 'SibSp',
       'Parch', 'age_scaled', 'Pclass_1',
        'embarked_C', 'embarked_Q','embarked_S',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'fare_scaled']

In [16]:
df_train_final = df_train[keep_columns]
df_train_final.head()

,Survived,Pclass,SibSp,Parch,age_scaled,Pclass_1,embarked_C,embarked_Q,embarked_S,Pclass_2,Pclass_3,Sex_female,Sex_male,fare_scaled
0,0,3,1,0,-0.592481,0,0,0,1,0,1,0,1,-0.502445
1,1,1,1,0,0.638789,1,1,0,0,0,0,1,0,0.786845
2,1,3,0,0,-0.284663,0,0,0,1,0,1,1,0,-0.488854
3,1,1,1,0,0.407926,1,0,0,1,0,0,1,0,0.420730
4,0,3,0,0,0.407926,0,0,0,1,0,1,0,1,-0.486337


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,roc_auc_score, roc_curve

In [32]:
train_data_X = df_train_final.drop(['Survived'], axis=1)
train_data_y = df_train_final['Survived']
print(train_data_X.shape)
print(train_data_y.shape)

(891, 13)
(891,)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.3, random_state=10)

In [69]:
def get_top_n_features(X_train, y_train, top_n_features):
    rf = RandomForestClassifier(random_state=0)
    rf_params = {'n_estimators': [50, 300], 'min_samples_split': [2, 3], 'max_depth': [5, 10]}
    rf_best = GridSearchCV(rf, rf_params, n_jobs=1, cv=10, verbose=1)
    rf_best.fit(X_train, y_train)
    print('Best RF Params:' + str(rf_best.best_params_))
    print('Best RF Score:' + str(rf_best.best_score_))
    print('RF Train Score:' + str(rf_best.score(X_train, y_train)))
    feature_imp_sorted_rf = pd.DataFrame({'feature': list(X_train),
                                          'importance': rf_best.best_estimator_.feature_importances_}).sort_values('importance', ascending=False)
    print(feature_imp_sorted_rf)
    features_top_n_rf = feature_imp_sorted_rf.head(top_n_features)['feature']
    print('Sample 10 Features from RF Classifier')
    print(str(features_top_n_rf[:10]))
    
    ada =AdaBoostClassifier(random_state=0)
    ada_params = {'n_estimators': [50, 150, 300], 'learning_rate': [0.01, 0.1]}
    ada_best = GridSearchCV(ada, ada_params, n_jobs=1, cv=10, verbose=1)
    ada_best.fit(X_train, y_train)
    print('Best Ada Params:' + str(ada_best.best_params_))
    print('Best Ada Score:' + str(ada_best.best_score_))
    print('Ada Train Score:' + str(ada_best.score(X_train, y_train)))
    feature_imp_sorted_ada = pd.DataFrame({'feature': list(X_train),
                                           'importance': ada_best.best_estimator_.feature_importances_}).sort_values('importance', ascending=False)
    print(feature_imp_sorted_ada)
    features_top_n_ada = feature_imp_sorted_ada.head(top_n_features)['feature']
    print('Sample 10 Feature from Ada Classifier:')
    print(str(features_top_n_ada[:10]))
    
    # GradientBoostingClassifier , DecisionTreeClassifier & others
    
    # merge
    features_top_n = pd.concat([features_top_n_rf, features_top_n_ada], 
                               ignore_index=True).drop_duplicates()
    features_importance = pd.concat([feature_imp_sorted_rf, feature_imp_sorted_ada],ignore_index=True)
    return features_top_n , features_importance

In [70]:
features_top_n, features_importance = get_top_n_features(X_train, y_train, 10)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   19.7s finished


Best RF Params:{'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 3}
Best RF Score:0.812199036918138
RF Train Score:0.9357945425361156
        feature  importance
12  fare_scaled    0.210995
3    age_scaled    0.204146
11     Sex_male    0.170347
10   Sex_female    0.156971
1         SibSp    0.058437
9      Pclass_3    0.046250
2         Parch    0.042442
0        Pclass    0.029850
4      Pclass_1    0.026239
7    embarked_S    0.020116
5    embarked_C    0.019344
8      Pclass_2    0.008887
6    embarked_Q    0.005976
Sample 10 Features from RF Classifier
12    fare_scaled
3      age_scaled
11       Sex_male
10     Sex_female
1           SibSp
9        Pclass_3
2           Parch
0          Pclass
4        Pclass_1
7      embarked_S
Name: feature, dtype: object
Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   14.2s finished


Best Ada Params:{'n_estimators': 300, 'learning_rate': 0.01}
Best Ada Score:0.797752808988764
Ada Train Score:0.8009630818619583
        feature  importance
11     Sex_male    0.286667
10   Sex_female    0.240000
0        Pclass    0.180000
1         SibSp    0.156667
4      Pclass_1    0.073333
9      Pclass_3    0.036667
5    embarked_C    0.013333
12  fare_scaled    0.013333
2         Parch    0.000000
3    age_scaled    0.000000
6    embarked_Q    0.000000
7    embarked_S    0.000000
8      Pclass_2    0.000000
Sample 10 Feature from Ada Classifier:
11       Sex_male
10     Sex_female
0          Pclass
1           SibSp
4        Pclass_1
9        Pclass_3
5      embarked_C
12    fare_scaled
2           Parch
3      age_scaled
Name: feature, dtype: object


In [71]:
print(features_top_n)
print(features_importance)

0     fare_scaled
1      age_scaled
2        Sex_male
3      Sex_female
4           SibSp
5        Pclass_3
6           Parch
7          Pclass
8        Pclass_1
9      embarked_S
16     embarked_C
Name: feature, dtype: object
        feature  importance
0   fare_scaled    0.210995
1    age_scaled    0.204146
2      Sex_male    0.170347
3    Sex_female    0.156971
4         SibSp    0.058437
5      Pclass_3    0.046250
6         Parch    0.042442
7        Pclass    0.029850
8      Pclass_1    0.026239
9    embarked_S    0.020116
10   embarked_C    0.019344
11     Pclass_2    0.008887
12   embarked_Q    0.005976
13     Sex_male    0.286667
14   Sex_female    0.240000
15       Pclass    0.180000
16        SibSp    0.156667
17     Pclass_1    0.073333
18     Pclass_3    0.036667
19   embarked_C    0.013333
20  fare_scaled    0.013333
21        Parch    0.000000
22   age_scaled    0.000000
23   embarked_Q    0.000000
24   embarked_S    0.000000
25     Pclass_2    0.000000


In [73]:
rf_clf = RandomForestClassifier(n_estimators=300, max_depth=10, 
                            min_samples_split=3, n_jobs=1, verbose=1)
rf_clf.fit(X_train, y_train)
predictions = rf_clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.8246268656716418


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [74]:
ada_clf = AdaBoostClassifier(n_estimators=300, learning_rate=0.01)
ada_clf.fit(X_train, y_train)
predictions = ada_clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7947761194029851


In [83]:
svm = SVC(kernel="linear", C=0.025)
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)
print(accuracy_score(y_test, predictions))

0.8059701492537313


**可以看到准确率只能达到80%，这可能和特征处理有很大的关系。Age我们直接填充了均值；票价分桶效果是不是更好；丢弃了name, ticket等信息，这里应该还可以挖掘，比如人名中称谓；以及构造更多的特征。这里也足以看出特产对结果的影响。有名的一句话是Garbage in, garbage out。特征决定了上限。在实际工作中我们需要理解业务背后的故事，以及花更多的时间处理特征。**

##### 模型融合，to be continued...